In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install nltk
!pip install gensim

In [ ]:
import os
os.chdir("/content/drive/My Drive/colab/ESWC")
!ls

ECER.ipynb  ground_truth.csv  Irish_populated_places.csv  triples.tsv


In [ ]:
import pandas as pd
import pykeen
import numpy as np
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from sklearn.metrics.pairwise import cosine_similarity
import concurrent.futures
from math import log
import re
import gensim
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
import transformers
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import torch.nn.functional as F
from torch.utils.data import DataLoader
import requests
import time
import requests, tempfile
from io import BytesIO
from transformers import pipeline
import ast

INFO:pykeen.utils:Using opt_einsum
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
df = pd.read_csv('./triples.tsv', sep="\t", names=['head','relation','tail'])

In [ ]:
df.head()

,head,relation,tail
0,"http://dbpedia.org/resource/Holywell,_Swords",rdfs:label,"Holywell, Swords"
1,"http://dbpedia.org/resource/Holywell,_Swords",dbo:abstract,Holywell (Irish: Tobar Naofa) is a neighbourho...
2,"http://dbpedia.org/resource/Holywell,_Swords",dbo:country,http://dbpedia.org/resource/Republic_of_Ireland
3,"http://dbpedia.org/resource/Holywell,_Swords",dcterms:subject,http://dbpedia.org/resource/Category:Neighbour...
4,http://dbpedia.org/resource/Category:Neighbour...,skos:broader,"http://dbpedia.org/resource/Category:Swords,_D..."


In [ ]:
populated_places = pd.read_csv('Irish_populated_places.csv')['place'].unique()

In [ ]:
def text_clear(text):
    if not isinstance(text, str):
        return ""

    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
def generate_embeddings(triples_file, target_entities, model_name='RotatE'):

    df = pd.read_csv(triples_file, sep="\t", names=['head','relation','tail'])


    direct_triples = df[df['head'].isin(target_entities) | df['tail'].isin(target_entities)]
    subjects = set(direct_triples['tail'][direct_triples['relation'] == 'dcterms:subject'])
    broader_subjects = set(df['tail'][df['head'].isin(subjects) & (df['relation'] == 'skos:broader')])

    all_subjects = subjects.union(broader_subjects)
    subject_triples = df[df['head'].isin(all_subjects) | df['tail'].isin(all_subjects)]
    type_relations = df[(df['relation'] == 'rdf:type') & (df['head'].isin(direct_triples['tail']))]


    combined_triples = pd.concat([direct_triples, subject_triples, type_relations]).drop_duplicates()

    if combined_triples.empty:
        print("No relevant triples found for the given entities.")
        return None


    triples_array = combined_triples[['head', 'relation', 'tail']].values
    print(f"Number of related triples found: {len(triples_array)}")
    print(triples_array)


    triples_factory = TriplesFactory.from_labeled_triples(triples_array)

    model_kwargs = dict(embedding_dim=100)
    if model_name == 'TransH':
        model_kwargs['relation_dim'] = 50  # TransH requires relation_dim

    result = pykeen.pipeline.pipeline(
        model=model_name,
        training=triples_factory,
        validation=triples_factory,
        testing=triples_factory,
        training_kwargs=dict(num_epochs=300, use_tqdm_batch=False),
        optimizer_kwargs=dict(lr=0.1),
        model_kwargs=model_kwargs,
    )


    model = result.model

    entity_embeddings = model.entity_representations[0](indices=None).detach().cpu().numpy()

    id_to_entity = {v: k for k, v in triples_factory.entity_to_id.items()}

    entity_embedding_dict = {}

    for entity in target_entities:
        entity_id = triples_factory.entity_to_id.get(entity)
        if entity_id is not None:
            entity_embedding = entity_embeddings[entity_id]
            entity_embedding_dict[entity] = entity_embedding
            print(f"Embedding for {entity}: {entity_embedding[:5]}...")
        else:
            print(f"Entity {entity} not found in the trained model.")

    return entity_embedding_dict


entity_embeddings = generate_embeddings("triples.tsv", populated_places, model_name='RotatE')

INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


Number of related triples found: 2825
[['http://dbpedia.org/resource/Holywell,_Swords' 'rdfs:label'
  'Holywell, Swords']
 ['http://dbpedia.org/resource/Holywell,_Swords' 'dbo:abstract'
  "Holywell (Irish: Tobar Naofa) is a neighbourhood near Swords, Dublin, Ireland. Developed and marketed in phases and under various names, Feltrim Hall, Gorse Hill, Abbey Stone, Holywell and The Meadows, all the street names in the community include the common name Holywell e.g. 'Holywell Drive'. It is the eastern part of the census town of Kinsealy–Drinan, separated from the western part by the M1 motorway."]
 ['http://dbpedia.org/resource/Holywell,_Swords' 'dbo:country'
  'http://dbpedia.org/resource/Republic_of_Ireland']
 ...
 ['http://dbpedia.org/resource/City_status_in_Ireland' 'rdf:type'
  'http://dbpedia.org/ontology/Place']
 ['http://dbpedia.org/resource/Capital_city' 'rdf:type'
  'http://dbpedia.org/ontology/City']
 ['http://dbpedia.org/resource/Capital_city' 'rdf:type'
  'http://dbpedia.org/o

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Training epochs on cpu:   0%|          | 0/300 [00:00<?, ?epoch/s]

Evaluating on cpu:   0%|          | 0.00/2.83k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 36.94s seconds


Embedding for http://dbpedia.org/resource/Holywell,_Swords: [ 0.3878326-3.5979805j -1.4823203-0.8420464j -3.7639697-3.4339185j
 -1.807115 +6.872142j  -2.859122 +5.8816166j]...
Embedding for http://dbpedia.org/resource/Leyny: [ 1.3448514+2.2502613j   4.6601806-1.7279546j  -4.1381917-5.6395135j
 -1.4578998+0.43353707j -2.3685403-0.3463325j ]...
Embedding for http://dbpedia.org/resource/County_Cavan: [ 3.1017869 +0.29470113j  0.41324794-0.20239422j -4.3394365 -2.0539525j
 -1.2181296 +3.8780494j   1.0689627 +2.5629668j ]...
Embedding for http://dbpedia.org/resource/County_Cork: [ 1.7761132-3.740342j    3.869099 -0.75967604j -0.7113023-4.6583986j
 -4.536837 +2.680913j   -2.2325814+3.156667j  ]...
Embedding for http://dbpedia.org/resource/County_Galway: [ 1.6586564+2.1170046j -1.6266274-3.8557072j  4.0602345-4.7523327j
 -3.7942512+2.6740787j  2.5025535+1.0717264j]...
Embedding for http://dbpedia.org/resource/County_Kerry: [ 1.8882438 +0.597556j   -2.1289034 +3.1826673j  -0.09364926+3.7458196

In [ ]:
unstructured_data = pd.read_csv('Irish_populated_places.csv')

In [ ]:
unstructured_data['subject'] = unstructured_data['subject'].str.replace(
    'http://dbpedia.org/resource/Category:', '', regex=False
).map(lambda x: x.replace('_',' '))

In [ ]:
unstructured_data['broaderSubject'] = unstructured_data['broaderSubject'].fillna('').str.replace(
    'http://dbpedia.org/resource/Category:', '', regex=False
).str.replace('_', ' ')

In [ ]:
unstructured_data['country'] = unstructured_data['country'].fillna('').str.replace(
    'http://dbpedia.org/resource/', '', regex=False
).str.replace('_', ' ')

In [ ]:
unstructured_data['placeType'] = unstructured_data['placeType'].fillna('').str.replace(
    'http://dbpedia.org/resource/', '', regex=False
).str.replace('_', ' ')

In [ ]:
unstructured_data['broaderPlaceType'] = unstructured_data['broaderPlaceType'].fillna('').str.replace(
    'http://dbpedia.org/ontology/', '', regex=False
)

In [ ]:
columns_to_clean = ['placeName', 'abstract', 'country', 'subject', 'broaderSubject', 'placeType', 'broaderPlaceType']
for col in columns_to_clean:
    unstructured_data[col] = unstructured_data[col].apply(text_clear)

In [ ]:
unstructured_data

,place,placeName,abstract,country,subject,broaderSubject,placeType,broaderPlaceType,population
0,"http://dbpedia.org/resource/Holywell,_Swords",holywell swords,holywell is a neighbourhood near swords dublin...,republic of ireland,neighbourhoods in swords dublin,swords dublin,swords dublin,place,2479
1,"http://dbpedia.org/resource/Holywell,_Swords",holywell swords,holywell is a neighbourhood near swords dublin...,republic of ireland,neighbourhoods in swords dublin,swords dublin,swords dublin,location,2479
2,"http://dbpedia.org/resource/Holywell,_Swords",holywell swords,holywell is a neighbourhood near swords dublin...,republic of ireland,neighbourhoods in swords dublin,swords dublin,swords dublin,populated place,2479
3,"http://dbpedia.org/resource/Holywell,_Swords",holywell swords,holywell is a neighbourhood near swords dublin...,republic of ireland,neighbourhoods in swords dublin,swords dublin,swords dublin,settlement,2479
4,"http://dbpedia.org/resource/Holywell,_Swords",holywell swords,holywell is a neighbourhood near swords dublin...,republic of ireland,neighbourhoods in swords dublin,swords dublin,swords dublin,town,2479
...,...,...,...,...,...,...,...,...,...
2035,http://dbpedia.org/resource/Rosses_Point_Penin...,rosses point peninsula,the rosses point peninsula is a small peninsul...,republic of ireland,maritime folklore,maritime culture,,,1580
2036,http://dbpedia.org/resource/Rosses_Point_Penin...,rosses point peninsula,the rosses point peninsula is a small peninsul...,republic of ireland,maritime folklore,nautical fiction,,,1580
2037,http://dbpedia.org/resource/Rosses_Point_Penin...,rosses point peninsula,the rosses point peninsula is a small peninsul...,republic of ireland,piracy in the atlantic ocean,history of the atlantic ocean,,,1580
2038,http://dbpedia.org/resource/Rosses_Point_Penin...,rosses point peninsula,the rosses point peninsula is a small peninsul...,republic of ireland,piracy in the atlantic ocean,piracy by body of water,,,1580


In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def miniLM_embeddings(text):
  embeddings = model.encode(text)

  return embeddings

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
abstract = unstructured_data[['place', 'abstract']].drop_duplicates(subset=['place'])

In [ ]:
abstract['abstract_embedding'] = abstract['abstract'].apply(miniLM_embeddings)

In [ ]:
unstructured_data = unstructured_data.merge(
    abstract[['place', 'abstract_embedding']],
    on='place',
    how='left'
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def Bert_embeddings(text):
  if pd.isna(text) or text == '':
      return torch.zeros(model.config.hidden_size)
  if isinstance(text, list):
      text = [str(item) for item in text if pd.notna(item) and item != '']
      if not text:
          return torch.zeros(model.config.hidden_size)
  elif isinstance(text, str):
      text = [text]
  else:
      return torch.zeros(model.config.hidden_size)

  encoding = tokenizer.batch_encode_plus(
      text,
      padding = True,
      truncation = True,
      return_tensors = 'pt',
      add_special_tokens=True

  )
  input_ids = encoding['input_ids']
  attention_mask = encoding['attention_mask']

  with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        token_embeddings = outputs.last_hidden_state  # (1, seq_len, hidden_dim)

        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        mean_embedding = sum_embeddings / sum_mask

  return mean_embedding.squeeze(0)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
unstructured_data['placeName_embedding'] = unstructured_data['placeName'].apply(Bert_embeddings)
unstructured_data['country_embedding'] = unstructured_data['country'].apply(Bert_embeddings)
unstructured_data['subject_embedding'] = unstructured_data['subject'].apply(Bert_embeddings)
unstructured_data['broaderSubject_embedding'] = unstructured_data['broaderSubject'].apply(Bert_embeddings)
unstructured_data['placeType_embedding'] = unstructured_data['placeType'].apply(Bert_embeddings)
unstructured_data['broaderPlaceType_embedding'] = unstructured_data['broaderPlaceType'].apply(Bert_embeddings)

In [ ]:
class ShortTermCombine(nn.Module):
    def __init__(self, term_dim, embed_dim):
        super().__init__()
        self.attn = nn.MultiheadAttention(term_dim, num_heads=2, batch_first=True)
        self.proj = nn.Linear(term_dim, embed_dim)

    def forward(self, term_embeddings):
        # term_embeddings: (batch, num_terms, term_dim)
        print("term embeddings",term_embeddings.shape)
        attn_out, _ = self.attn(term_embeddings, term_embeddings, term_embeddings)
        pooled = attn_out.mean(dim=1)
        return self.proj(pooled)        #(batch, embed_dim)

In [ ]:
class FiLMTextCombine(nn.Module):
    def __init__(self, abs_dim, term_dim, embed_dim):
        super().__init__()

        self.abs_proj = nn.Linear(abs_dim, embed_dim)

        self.gate_layer = nn.Sequential(
            nn.Linear(embed_dim * 2, embed_dim),
            nn.Sigmoid()
        )

        # FiLM parameters from numeric input (e.g., population)
        self.film_gen = nn.Linear(1, embed_dim * 2)  # outputs gamma and beta

    def forward(self, emb_abs, emb_term, number):
        print("number:", number.shape)

        number = torch.log1p(number)
        number = number / (number.max() + 1e-8)


        abs_proj = self.abs_proj(emb_abs)           # (B, D)
        term_proj = emb_term        # (B, D)


        gate_input = torch.cat([abs_proj, term_proj], dim=1)
        gate = self.gate_layer(gate_input)
        text_fused = gate * abs_proj + (1 - gate) * term_proj  # (B, D)


        film_params = self.film_gen(number)  # (B, 2D)
        gamma, beta = film_params.chunk(2, dim=1)  # each is (B, D)

        # Apply FiLM modulation
        modulated = gamma * text_fused + beta  # (B, D)

        return modulated

In [ ]:
class CrossAttentionF(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.query_proj = nn.Linear(dim, dim)
        self.key_proj = nn.Linear(dim, dim)
        self.value_proj = nn.Linear(dim, dim)
        self.scale = dim ** -0.5

    def forward(self, text_emb, kg_emb):
        # [B, D]
        query = self.query_proj(text_emb).unsqueeze(1)  # [B, 1, D]
        key = self.key_proj(kg_emb).unsqueeze(1)        # [B, 1, D]
        value = self.value_proj(kg_emb).unsqueeze(1)    # [B, 1, D]

        attn_weights = torch.matmul(query, key.transpose(-2, -1)) * self.scale
        attn_weights = torch.softmax(attn_weights, dim=-1)  # [B, 1, 1]

        context = torch.matmul(attn_weights, value)  # [B, 1, D]
        return context.squeeze(1)  # [B, D]

In [ ]:
class GatedMechanism(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.gate = nn.Linear(dim * 2, dim)
        self.output = nn.Linear(dim * 3, dim)

    def forward(self, text_feat, kg_feat):
        combined = torch.cat([text_feat, kg_feat], dim=1)  # [B, 2D]
        gate = torch.sigmoid(self.gate(combined))          # [B, D]
        fused = gate * text_feat + (1 - gate) * kg_feat    # [B, D]
        return self.output(torch.cat([fused, combined], dim=1))  # [B, 3D]

In [ ]:
class EntitySimilarityModel(nn.Module):
    def __init__(self, abs_dim=384, term_dim=768, embed_dim=100):
        super().__init__()
        self.short_terms_combine = ShortTermCombine(term_dim, embed_dim)
        self.text_combine = FiLMTextCombine(abs_dim, term_dim, embed_dim)  # updated
        self.cross_attention = CrossAttentionF(embed_dim)
        self.gated_mechanism = GatedMechanism(embed_dim)

        self.final_fc = nn.Sequential(
            nn.Linear(embed_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, e1_abs, e1_term, e1_num, e1_kg, e2_abs, e2_term, e2_num, e2_kg):
        # Combine text and population for each entity

        e1_terms = self.short_terms_combine(e1_term)
        e2_terms = self.short_terms_combine(e2_term)

        e1_text = self.text_combine(e1_abs, e1_terms, e1_num)
        e2_text = self.text_combine(e2_abs, e2_terms, e2_num)

        # Cross-attention with KG embeddings
        e1_cross = self.cross_attention(e1_text, e1_kg)
        e1_fused = self.gated_mechanism(e1_text, e1_cross)

        e2_cross = self.cross_attention(e2_text, e2_kg)
        e2_fused = self.gated_mechanism(e2_text, e2_cross)

        # Cosine similarity
        cos_sim = F.cosine_similarity(e1_fused, e2_fused, dim=1, eps=1e-8)
        return cos_sim.unsqueeze(1)

In [ ]:
class EntityPairDataset(torch.utils.data.Dataset):
    def __init__(self, data, text_emb, kg_emb, numeric_dict, term_dim=768):
        self.data = data
        self.text_emb = text_emb
        self.kg_emb = kg_emb
        self.numeric_dict = numeric_dict
        self.term_dim = term_dim

    def __len__(self):
        return len(self.data)

    def _stack_terms(self, e_text):
        terms = []
        for field in ['place_name', 'country', 'subject',
                      'broaderSubject', 'placeType', 'broaderPlaceType']:
            arrs = e_text[field]  # always a list
            if arrs:  # non-empty
                terms.append(np.stack(arrs, axis=0))
        if terms:
            return np.concatenate(terms, axis=0)  # (num_terms, term_dim)
        else:
            return np.zeros((1, self.term_dim), dtype=np.float32)

    def __getitem__(self, idx):
        e1, e2, score = self.data[idx]

        e1_text = self.text_emb[e1]
        e2_text = self.text_emb[e2]

        e1_terms = self._stack_terms(e1_text)
        e2_terms = self._stack_terms(e2_text)

        return (
            torch.tensor(np.stack(e1_text['abstract']), dtype=torch.float32).squeeze(0),
            torch.tensor(e1_terms, dtype=torch.float32),
            torch.tensor(self.numeric_dict[e1]['num'], dtype=torch.float32),
            torch.tensor(self.kg_emb[e1], dtype=torch.float32),

            torch.tensor(np.stack(e2_text['abstract']), dtype=torch.float32).squeeze(0),
            torch.tensor(e2_terms, dtype=torch.float32),
            torch.tensor(self.numeric_dict[e2]['num'], dtype=torch.float32),
            torch.tensor(self.kg_emb[e2], dtype=torch.float32),

            torch.tensor([score], dtype=torch.float32)
        )

In [ ]:
model = EntitySimilarityModel(embed_dim=100)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

In [ ]:
ground_truth = pd.read_csv("./ground_truth.csv")
y_true_dict = ground_truth.groupby('geographical_entity_1')['geographical_entity_2'].apply(list).to_dict()

In [ ]:
y_true_list = [
    (e1, e2, score)
    for e1, e2, score in ground_truth[['geographical_entity_1', 'geographical_entity_2', 'F-score']].values
]

In [ ]:
from collections import defaultdict

text_emb_dict = defaultdict(lambda: {
    'abstract': [],
    'place_name': [],
    'country': [],
    'subject': [],
    'broaderSubject': [],
    'placeType': [],
    'broaderPlaceType': []
})

for _, row in unstructured_data.iterrows():
    place = row['place']

    if not text_emb_dict[place]['abstract']:
        text_emb_dict[place]['abstract'] = [np.array(row['abstract_embedding'])]
    if not text_emb_dict[place]['place_name']:
        text_emb_dict[place]['place_name'] = [np.array(row['placeName_embedding'])]
    if not text_emb_dict[place]['country']:
        text_emb_dict[place]['country'] = [np.array(row['country_embedding'])]

    def append_unique(field, emb):
        if not any(np.array_equal(emb, e) for e in text_emb_dict[place][field]):
            text_emb_dict[place][field].append(emb)

    append_unique('subject', np.array(row['subject_embedding']))
    append_unique('broaderSubject', np.array(row['broaderSubject_embedding']))
    append_unique('placeType', np.array(row['placeType_embedding']))
    append_unique('broaderPlaceType', np.array(row['broaderPlaceType_embedding']))

In [ ]:
numeric_dict = {}

for _, row in unstructured_data.iterrows():
    place = row['place']
    num = np.array([row['population']], dtype=np.float32)
    numeric_dict[place] = {
        'num': num
    }

In [ ]:
numeric_dict_zeroed = {e: {'num': np.zeros_like(v['num'])} for e, v in numeric_dict.items()}

In [ ]:
kg_emb_dict = entity_embeddings

In [ ]:
kg_emb_dict_zeroed = {e: np.zeros_like(v) for e, v in kg_emb_dict.items()}

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    (e1_abs, e1_terms, e1_num, e1_kg,
     e2_abs, e2_terms, e2_num, e2_kg,
     scores) = zip(*batch)

    e1_abs = torch.stack(e1_abs)
    e1_num = torch.stack(e1_num)
    e1_kg  = torch.stack(e1_kg)

    e2_abs = torch.stack(e2_abs)
    e2_num = torch.stack(e2_num)
    e2_kg  = torch.stack(e2_kg)

    scores = torch.stack(scores)

    e1_terms = pad_sequence(e1_terms, batch_first=True)  # (batch, max_terms, term_dim)
    e2_terms = pad_sequence(e2_terms, batch_first=True)

    return e1_abs, e1_terms, e1_num, e1_kg, e2_abs, e2_terms, e2_num, e2_kg, scores

In [ ]:
# Build dataset and loader
train_dataset = EntityPairDataset(y_true_list, text_emb_dict, kg_emb_dict, numeric_dict_zeroed)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

for epoch in range(25):
    model.train()
    epoch_loss = 0
    for (e1_abs, e1_terms, e1_num, e1_kg,
         e2_abs, e2_terms, e2_num, e2_kg,
         score) in train_loader:

        # Forward pass with short terms
        pred = model(e1_abs, e1_terms, e1_num, e1_kg,
                     e2_abs, e2_terms, e2_num, e2_kg)

        # Compute loss
        loss = loss_fn(pred, score)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader):.4f}")

Streaming output truncated to the last 5000 lines.
term embeddings torch.Size([16, 24, 768])
term embeddings torch.Size([16, 25, 768])
number: torch.Size([16, 1])
number: torch.Size([16, 1])
term embeddings torch.Size([16, 29, 768])
term embeddings torch.Size([16, 26, 768])
number: torch.Size([16, 1])
number: torch.Size([16, 1])
term embeddings torch.Size([16, 30, 768])
term embeddings torch.Size([16, 24, 768])
number: torch.Size([16, 1])
number: torch.Size([16, 1])
term embeddings torch.Size([16, 24, 768])
term embeddings torch.Size([16, 21, 768])
number: torch.Size([16, 1])
number: torch.Size([16, 1])
term embeddings torch.Size([16, 23, 768])
term embeddings torch.Size([16, 27, 768])
number: torch.Size([16, 1])
number: torch.Size([16, 1])
term embeddings torch.Size([16, 38, 768])
term embeddings torch.Size([16, 21, 768])
number: torch.Size([16, 1])
number: torch.Size([16, 1])
term embeddings torch.Size([16, 17, 768])
term embeddings torch.Size([16, 21, 768])
number: torch.Size([16, 1

In [ ]:
def get_entity_representation(model, text_emb_dict, kg_emb_dict, numeric_dict, term_dim=768):
    """
    Build final fused representations for all entities.

    Args:
        model: trained EntitySimilarityModel
        text_emb_dict: dict of text embeddings per entity (lists of embeddings)
        kg_emb_dict: dict of KG embeddings per entity
        numeric_dict: dict of numeric features per entity
        term_dim: dimension of short-term embeddings (e.g., 768 for BERT)

    Returns:
        entity_repr: {entity: torch.Tensor (embed_dim,)}
    """
    model.eval()
    entity_repr = {}

    def stack_terms(e_text):
        terms = []
        for field in ['place_name', 'country', 'subject',
                      'broaderSubject', 'placeType', 'broaderPlaceType']:
            arrs = e_text[field]  # always a list
            if arrs:
                terms.append(np.stack(arrs, axis=0))
        if terms:
            return np.concatenate(terms, axis=0)  # (num_terms, term_dim)
        else:
            return np.zeros((1, term_dim), dtype=np.float32)

    with torch.no_grad():
        for entity in text_emb_dict:
            # abstract → always (1, abs_dim)
            abs_emb = torch.tensor(np.stack(text_emb_dict[entity]['abstract']),
                                   dtype=torch.float32)

            # short terms → (1, num_terms, term_dim)
            term_emb = stack_terms(text_emb_dict[entity])
            term_emb = torch.tensor(term_emb, dtype=torch.float32).unsqueeze(0)

            # numeric features
            num_emb = torch.tensor([numeric_dict[entity]['num']], dtype=torch.float32)

            # KG embedding
            kg_emb = torch.tensor(kg_emb_dict[entity], dtype=torch.float32).unsqueeze(0)

            # forward pass
            terms = model.short_terms_combine(term_emb)                    # (1, embed_dim)
            text_combined = model.text_combine(abs_emb, terms, num_emb)    # (1, embed_dim)
            cross = model.cross_attention(text_combined, kg_emb)           # (1, embed_dim)
            fused = model.gated_mechanism(text_combined, cross)            # (1, embed_dim)

            entity_repr[entity] = fused.squeeze(0).cpu()                   # (embed_dim,)

    return entity_repr

In [ ]:
def compute_similar_places(entity_repr, top_k=10):
    entities = list(entity_repr.keys())
    all_embeddings = torch.stack([entity_repr[e] for e in entities])  # [N, D]
    sims = F.cosine_similarity(all_embeddings.unsqueeze(1), all_embeddings.unsqueeze(0), dim=-1)

    similar_places_dict = {}
    for i, e in enumerate(entities):
        sim_scores = sims[i]
        topk = torch.topk(sim_scores, k=top_k+1)  # +1 to skip itself
        indices = topk.indices.tolist()
        similar_entities = [entities[j] for j in indices if j != i][:top_k]
        similar_places_dict[e] = similar_entities
    return similar_places_dict

In [ ]:
entity_repr = get_entity_representation(model, text_emb_dict, kg_emb_dict, numeric_dict)

term embeddings torch.Size([1, 10, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 8, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 18, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 16, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 16, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 16, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 16, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 19, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 16, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 16, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 16, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 16, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 19, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 16, 768])
number: torch.Size([1, 1])
term embeddings torch.Size([1, 13, 768])
number: 

In [ ]:
similar_places = compute_similar_places(entity_repr, top_k=10)

In [ ]:
def evluation(k, y_true_dict, similar_places_dict):
  invalid_places = []
  # Compute metrics
  precisions, recalls, ndcgs, hits, map_scores,mrrs = [], [], [], [], [], []
  place_idxs = list(y_true_dict.keys())

  for pid in place_idxs:
      if pid not in similar_places_dict or len(similar_places_dict[pid]) < k:
          invalid_places.append(pid)
          continue
      pred_list, rel_set = similar_places_dict[pid][:k], y_true_dict[pid]
      pred_real = "pid:"+str(pid)+' '+"pred_list:"+str(pred_list)+' '+"rel_set:"+str(rel_set)

      if len(pred_list) == 0:
          continue


      dcg = 0.0
      hit_num = 0.0
      for i in range(len(pred_list)):
          if pred_list[i] in rel_set:
              dcg += 1. / (log(i + 2) / log(2))
              hit_num += 1
      # idcg
      idcg = 0.0
      for i in range(min(len(rel_set), len(pred_list))):
          idcg += 1. / (log(i + 2) / log(2))
      ndcg = dcg / idcg
      recall = hit_num / len(rel_set)
      precision = hit_num / len(pred_list)
      hit = 1.0 if hit_num > 0.0 else 0.0

      #map
      map_score = 0.0
      num_hits = 0.0
      score = 0.0
      for i,p in enumerate(pred_list):
          if p in rel_set and p not in pred_list[:i]:
              num_hits+=1.0
              score+=num_hits/(i+1.0)
      map_score = score/min(len(rel_set),k)
      #map_score = score / min(len(rel_set), len(pred_list)) if len(rel_set) > 0 else 0.0

      #MRR
      rr = 0.0
      for i, p in enumerate(pred_list):
          if p in rel_set:
              rr = 1.0 / (i + 1.0)
              break

      ndcgs.append(ndcg)
      recalls.append(recall)
      precisions.append(precision)
      hits.append(hit)
      map_scores.append(map_score)
      mrrs.append(rr)

  avg_precision = np.mean(precisions) * 100
  avg_recall = np.mean(recalls) * 100
  avg_ndcg = np.mean(ndcgs) * 100
  avg_hit = np.mean(hits) * 100
  avg_map = np.mean(map_scores) * 100
  avg_mrr = np.mean(mrrs) * 100

  print("invalid places:", str(len(invalid_places)))
  print('MAP={:.3f} | NDCG={:.3f} |  Recall={:.3f} | Precision={:.3f} | Hits={:.3f} | MRR={:.3f}'.format(
          avg_map, avg_ndcg, avg_recall, avg_precision, avg_hit, avg_mrr))

In [ ]:
evluation(k=10, y_true_dict=y_true_dict, similar_places_dict=similar_places)

invalid places: 0
MAP=35.344 | NDCG=46.347 |  Recall=21.222 | Precision=41.843 | Hits=88.737 | MRR=70.851
